### Ví dụ 1: So sánh lưu trữ an toàn và không an toàn

Đây là ví dụ so sánh việc lưu trữ dữ liệu nhạy cảm (số an sinh xã hội - `ssn`) ở dạng văn bản thô (không an toàn) và dạng đã được mã hóa (an toàn).

In [1]:
from pymongo import MongoClient
from cryptography.fernet import Fernet

# --- Kết nối tới MongoDB (sử dụng user admin đã tạo trong docker-compose) ---
MONGO_ADMIN_USER = 'mongo_user'
MONGO_ADMIN_PASSWORD = 'mongo_password'
MONGO_HOST = 'localhost'
MONGO_PORT = 27018
MONGO_DB_DEMO = 'security_demo_db'

try:
    client = MongoClient(host=MONGO_HOST, port=MONGO_PORT, username=MONGO_ADMIN_USER, password=MONGO_ADMIN_PASSWORD)
    client.admin.command('ismaster') # Check connection
    db = client[MONGO_DB_DEMO]
    print("✅ Kết nối MongoDB thành công!")
except Exception as e:
    print(f"❌ Kết nối MongoDB thất bại: {e}")

# --- 1. TRƯỜNG HỢP KHÔNG BẢO MẬT (Lưu văn bản thuần) ---
print("\n--- 1. TRƯỜNG HỢP KHÔNG BẢO MẬT ---\n")
insecure_user_data = {"username": "user_insecure", "ssn": "123-456-7890"}
db.users_clear.drop()
db.users_clear.insert_one(insecure_user_data)
retrieved_insecure = db.users_clear.find_one({"username": "user_insecure"})
print(f"Dữ liệu đọc được từ DB: {retrieved_insecure}")
print(f"❗️ Dữ liệu nhạy cảm (SSN) đã bị lộ: {retrieved_insecure['ssn']}\n")

# --- 2. TRƯỜNG HỢP CÓ BẢO MẬT (Áp dụng mã hóa) ---
print("\n--- 2. TRƯỜNG HỢP CÓ BẢO MẬT ---\n")
key = Fernet.generate_key()
cipher_suite = Fernet(key)
secure_user_data = {"username": "user_secure", "ssn": "987-654-3210"}
encrypted_ssn = cipher_suite.encrypt(secure_user_data['ssn'].encode('utf-8'))
document_to_store = secure_user_data.copy()
document_to_store['ssn'] = encrypted_ssn
db.users_encrypted.drop()
db.users_encrypted.insert_one(document_to_store)
retrieved_secure = db.users_encrypted.find_one({"username": "user_secure"})
print(f"Dữ liệu đọc được từ DB: {retrieved_secure}")
print("✅ Dữ liệu nhạy cảm đã được mã hóa, không thể đọc trực tiếp.\n")
decrypted_ssn_bytes = cipher_suite.decrypt(retrieved_secure['ssn'])
decrypted_ssn = decrypted_ssn_bytes.decode('utf-8')
print(f"Dữ liệu SSN sau khi giải mã để sử dụng: {decrypted_ssn}")

✅ Kết nối MongoDB thành công!

--- 1. TRƯỜNG HỢP KHÔNG BẢO MẬT ---

Dữ liệu đọc được từ DB: {'_id': ObjectId('691d939b8c7275269f06c3b4'), 'username': 'user_insecure', 'ssn': '123-456-7890'}
❗️ Dữ liệu nhạy cảm (SSN) đã bị lộ: 123-456-7890


--- 2. TRƯỜNG HỢP CÓ BẢO MẬT ---

Dữ liệu đọc được từ DB: {'_id': ObjectId('691d939b8c7275269f06c3b5'), 'username': 'user_secure', 'ssn': b'gAAAAABpHZObUfKo4B1mmYVbzvJkSsOvffpe3wFujAZIquFlVt4tXQMOfb1iJRJTkGssW9r3kHQIgQ1aGM1MPBuum9xA64bSFQ=='}
✅ Dữ liệu nhạy cảm đã được mã hóa, không thể đọc trực tiếp.

Dữ liệu SSN sau khi giải mã để sử dụng: 987-654-3210


### Ví dụ 2: So sánh truy vấn an toàn và không an toàn (NoSQL Injection)

Toán tử `$where` trong MongoDB cho phép thực thi JavaScript phía server. Nếu sử dụng không cẩn thận bằng cách ghép chuỗi đầu vào từ người dùng, nó sẽ tạo ra lỗ hổng NoSQL Injection.

In [3]:
# --- Chuẩn bị dữ liệu --- #
db.login_users.drop()
db.login_users.insert_many([
    {"username": "admin", "password": "supersecretpassword_dkj398&(*&"},
    {"username": "guest", "password": "guest123"}
])
print("Đã tạo dữ liệu người dùng cho demo login.\n")

# --- 1. TRƯỜNG HỢP KHÔNG BẢO MẬT (Sử dụng $where) ---
print("--- 1. TRƯỜNG HỢP KHÔNG BẢO MẬT ---\n")

def unsafe_login(username, password):
    print(f"Thử đăng nhập với username: '{username}' và password: '{password}'")
    js_query = f"function() {{ return this.username == '{username}' && this.password == '{password}'; }}"
    print(f"Câu truy vấn $where sẽ thực thi: {js_query}")
    try:
        user = db.login_users.find_one({"$where": js_query})
        if user:
            print(f"---> ✅ Đăng nhập thành công! Xin chào, {user['username']}.\n")
        else:
            print("--> ❌ Đăng nhập thất bại.\n")
    except Exception as e:
        print(f"---> ⚠️ Lỗi khi thực thi $where: {e}\n")

# Kịch bản tấn công: Đăng nhập vào 'admin' mà không cần biết mật khẩu
malicious_password_input = "' || true || '"
unsafe_login("admin", malicious_password_input)

# --- 2. TRƯỜNG HỢP CÓ BẢO MẬT (Không dùng $where) ---
print("\n--- 2. TRƯỜNG HỢP CÓ BẢO MẬT ---\n")

def safe_login(username, password):
    print(f"Thử đăng nhập an toàn với username: '{username}' và password: '{password}'")
    query = {"username": username, "password": password}
    print(f"Câu truy vấn BSON an toàn: {query}")
    user = db.login_users.find_one(query)
    if user:
        print(f"---> ✅ Đăng nhập thành công! Xin chào, {user['username']}.\n")
    else:
        print("--> ❌ Đăng nhập thất bại.\n")

# Thử lại kịch bản tấn công với hàm an toàn
safe_login("admin", malicious_password_input)

Đã tạo dữ liệu người dùng cho demo login.

--- 1. TRƯỜNG HỢP KHÔNG BẢO MẬT ---

Thử đăng nhập với username: 'admin' và password: '' || true || ''
Câu truy vấn $where sẽ thực thi: function() { return this.username == 'admin' && this.password == '' || true || ''; }
---> ✅ Đăng nhập thành công! Xin chào, admin.


--- 2. TRƯỜNG HỢP CÓ BẢO MẬT ---

Thử đăng nhập an toàn với username: 'admin' và password: '' || true || ''
Câu truy vấn BSON an toàn: {'username': 'admin', 'password': "' || true || '"}
--> ❌ Đăng nhập thất bại.

